## Early stopping callback

### stop if objective function is below a certain threshold

In [13]:
from mango import Tuner

param_dict = dict(x=range(-10, 10))

def objfunc(p_list):
    return [p['x'] ** 2 for p in p_list]

def early_stop(results):
    '''
        stop if best objective is below 2
        results: dict (same keys as dict returned by tuner.minimize/maximize)
    '''
    return results['best_objective'] <= 2

config = dict(early_stopping=early_stop)

tuner = Tuner(param_dict, objfunc, conf_dict=config)
results = tuner.minimize()
results

  0%|          | 0/20 [00:00<?, ?it/s]

{'random_params': array([{'x': -6}, {'x': -2}], dtype=object),
 'random_params_objective': array([-36,  -4]),
 'params_tried': array([{'x': -6}, {'x': -2}, {'x': -3}, {'x': 3}, {'x': 0}], dtype=object),
 'objective_values': array([36,  4,  9,  9,  0]),
 'surrogate_values': array([-36.,  -4.,   0.,   0.,   0.]),
 'best_objective': 0,
 'best_params': {'x': 0}}

### stop if objective function does not improve for n iterations

In [14]:
from mango import Tuner

param_dict = dict(x=range(-10, 10))

def objfunc(p_list):
    return [p['x'] ** 2 for p in p_list]
    

def early_stop(results):
    '''
        stop if best objective does not improve for 2 iterations
        results: dict (same keys as dict returned by tuner.minimize/maximize)
    '''
    current_best = results['best_objective']
    patience_window = results['objective_values'][-3:]
    return min(patience_window) > current_best

config = dict(early_stopping=early_stop)

tuner = Tuner(param_dict, objfunc, conf_dict=config)
results = tuner.minimize()
results

  0%|          | 0/20 [00:00<?, ?it/s]

{'random_params': array([{'x': 7}, {'x': -4}], dtype=object),
 'random_params_objective': array([-49, -16]),
 'params_tried': array([{'x': 7}, {'x': -4}, {'x': -2}, {'x': -4}, {'x': -1}, {'x': -10},
        {'x': 6}, {'x': -1}, {'x': 6}, {'x': 2}, {'x': 0}, {'x': 1},
        {'x': 3}, {'x': -9}], dtype=object),
 'objective_values': array([ 49,  16,   4,  16,   1, 100,  36,   1,  36,   4,   0,   1,   9,
         81]),
 'surrogate_values': array([-4.90000000e+01, -1.60000000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.25445372e+00, -4.75155392e-02, -1.19646246e+00,
        -8.63547743e+00, -8.95304193e+01]),
 'best_objective': 0,
 'best_params': {'x': 0}}

### stop if objective function does not improve for n secs

In [33]:
import time
import numpy as np

from mango import Tuner

param_dict = dict(x=range(-10, 10))

def objfunc(p_list):
    time.sleep(0.5)
    return [p['x'] ** 2 for p in p_list]
    

class context:
    previous_best = None
    previous_best_time = None
    min_improvement_secs = 0.1
    

def early_stop(results):
    '''
        stop if objective does not improve for 0.1 seconds
    '''
    current_best = results['best_objective']
    current_time = time.time()
    
    _stop = False

    if context.previous_best is None:
        context.previous_best = current_best
        context.previous_best_time = current_time
        
    elif current_best == context.previous_best and \
            (current_time - context.previous_best_time > context.min_improvement_secs):
        print("no improvement in %f seconds: stopping early." % context.min_improvement_secs)
        _stop = True
        
    else:
        context.previous_best = current_best
        context.previous_best_time = current_time

    return _stop

config = dict(early_stopping=early_stop)

tuner = Tuner(param_dict, objfunc, conf_dict=config)
results = tuner.minimize()
results

  0%|          | 0/20 [00:00<?, ?it/s]

no improvement in 0.100000 seconds: stopping early.


{'random_params': array([{'x': 5}, {'x': -3}], dtype=object),
 'random_params_objective': array([-25,  -9]),
 'params_tried': array([{'x': 5}, {'x': -3}, {'x': 4}, {'x': 9}], dtype=object),
 'objective_values': array([25,  9, 16, 81]),
 'surrogate_values': array([-25.,  -9.,   0.,   0.]),
 'best_objective': 9,
 'best_params': {'x': -3}}